In [1]:
import os
import subprocess
import numpy as np
from tqdm import tqdm


LLVM_BINS = '/home/flint/.local/share/compiler_gym/llvm-v0/bin'
MCA_BIN = os.path.join(LLVM_BINS, 'llvm-mca')
# CLANG_BIN = os.path.join(LLVM_BINS, 'clang')
CLANG_BIN = 'clang'
LLC_BIN = os.path.join(LLVM_BINS, 'llc')
JOTAI_BENCHMARKS_DIRS = [
    "/home/flint/diplom/jotai-benchmarks/benchmarks/anghaLeaves/",
    "/home/flint/diplom/jotai-benchmarks/benchmarks/anghaMath/",
]
TMP_DIR = 'tmp'

O0 = '-O0'
O1 = '-O1'
O2 = '-O2'
O3 = '-O3'
OZ = '-Oz'

In [16]:
def get_mca_result(source_path, optimization):
    proc = subprocess.run(
        # f"{CLANG_BIN} {source_path} {optimization} --target=x86_64 -S -fsanitize=address,undefined,signed-integer-overflow -fno-sanitize-recover=all -o - | llvm-mca",
        f"{CLANG_BIN} {source_path} {optimization} --target=x86_64 -S -o - | llvm-mca",
        capture_output=True,
        shell=True,
        encoding="utf-8",
    )
    assert proc.returncode == 0, proc.stderr
    return proc.stdout


def get_mca_result_from_ir(ir):
    proc = subprocess.run(
        f"{LLC_BIN} | llvm-mca",
        input=ir,
        capture_output=True,
        shell=True,
        encoding="utf-8",
    )
    assert proc.returncode == 0, proc.stderr
    return proc.stdout


def parse_rblock_throughput(mca_output):
    line = mca_output.split('\n', maxsplit=9)[8]
    # print(line)
    return float(line.split()[2])

def get_rblock_throughput(source_path, optimization):
    return parse_rblock_throughput(get_mca_result(source_path, optimization))


In [3]:
jotai_benchmarks = []
for path in JOTAI_BENCHMARKS_DIRS:
    jotai_benchmarks.extend(
        [os.path.join(path, file) for file in os.listdir(path)]
    )

len(jotai_benchmarks)

31417

In [4]:
import compiler_gym                      # imports the CompilerGym environments
env = compiler_gym.make(                 # creates a new environment (same as gym.make)
    "llvm-v0",                           # selects the compiler to use
    observation_space="Autophase",       # selects the observation space
    reward_space="IrInstructionCountOz", # selects the optimization target
)

In [5]:
benchmark = next(env.datasets["benchmark://jotaibench-v0"].benchmarks())
env.reset(benchmark)

Installing the benchmark://jotaibench-v0 dataset. This may take a few moments ...


array([  0,   0,   8,   7,   1,   8,   3,   1,   1,   3,   0,   0,   0,
        17,   0,  10,   9,   0,  14,  40,  14,  12,  21,   8,  16,   0,
         6,  10,   0,   3,  14,  12,  10,  23,   7,   2,   0,  25,   7,
         0,   0,   6,   5,   0,   0,  17,   1,   1,   0,   0,  17, 140,
        82,  13,   0,  54])

In [11]:
benchmark

benchmark://jotaibench-v0/extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final

In [10]:
env.observation["Ir"]

'; ModuleID = \'-\'\nsource_filename = "-"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-unknown-linux-gnu"\n\n%struct._IO_FILE = type { i32, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, %struct._IO_marker*, %struct._IO_FILE*, i32, i32, i64, i16, i8, [1 x i8], i8*, i64, %struct._IO_codecvt*, %struct._IO_wide_data*, %struct._IO_FILE*, i8*, i64, i32, [20 x i8] }\n%struct._IO_marker = type opaque\n%struct._IO_codecvt = type opaque\n%struct._IO_wide_data = type opaque\n%struct.TYPE_3__ = type { i64, i64 }\n\n@rand_primes = dso_local constant [25 x i32] [i32 179, i32 103, i32 479, i32 647, i32 229, i32 37, i32 271, i32 557, i32 263, i32 607, i32 18743, i32 50359, i32 21929, i32 48757, i32 98179, i32 12907, i32 52937, i32 64579, i32 49957, i32 52567, i32 507163, i32 149939, i32 412157, i32 680861, i32 757751], align 16\n@next_i.counter = internal global i32 0, align 4\n@next_f.counter = internal global i32 0, 

In [17]:
parse_rblock_throughput(get_mca_result_from_ir(env.observation["Ir"]))

31.0

In [14]:
get_rblock_throughput(
    "/home/flint/diplom/jotai-benchmarks/benchmarks/anghaLeaves/extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final.c",
    O0,
)

139.0

In [28]:
!clang -S -emit-llvm /home/flint/diplom/jotai-benchmarks/benchmarks/anghaLeaves/extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final.c 
!cat extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final.ll
# !cat /home/flint/diplom/jotai-benchmarks/benchmarks/anghaLeaves/extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final.c

; ModuleID = '/home/flint/diplom/jotai-benchmarks/benchmarks/anghaLeaves/extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final.c'
source_filename = "/home/flint/diplom/jotai-benchmarks/benchmarks/anghaLeaves/extr_anypixelfirmwarecontrollersrcfifo.c_FIFO_available_Final.c"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

%struct.TYPE_3__ = type { i64, i64 }

@rand_primes = dso_local constant [25 x i32] [i32 179, i32 103, i32 479, i32 647, i32 229, i32 37, i32 271, i32 557, i32 263, i32 607, i32 18743, i32 50359, i32 21929, i32 48757, i32 98179, i32 12907, i32 52937, i32 64579, i32 49957, i32 52567, i32 507163, i32 149939, i32 412157, i32 680861, i32 757751], align 16
@.str = private unnamed_addr constant [3 x i8] c"%s\00", align 1
@.str.1 = private unnamed_addr constant [118 x i8] c"Usage:\0A    prog [ARGS]\0A\0AARGS:\0A       0            big-arr\0A       1            big-arr-10x\0A       2      

In [33]:
env.benchmark.source

'// ========================================================================= //\n\n// includes\n#include "stdio.h"\n#include "stdlib.h"\n#include "time.h"\n#include "string.h"\n#include "limits.h"\n#include "float.h"\n\n\n\n#define JOTAI_NUM_RANDS_ 25\n\nconst unsigned rand_primes[JOTAI_NUM_RANDS_] = {179, 103, 479, 647, 229, 37, 271, 557, 263, 607, 18743, 50359, 21929, 48757, 98179, 12907, 52937, 64579, 49957, 52567, 507163, 149939, 412157, 680861, 757751};\n\nint next_i() {\n  static counter = 0;\n  return (-2 * (counter % 2) + 1) * rand_primes[(++counter)%JOTAI_NUM_RANDS_];\n}\n\nfloat next_f() {\n  static counter = 0;\n  return rand_primes[(++counter)%JOTAI_NUM_RANDS_] / 757751.0F;\n} \n\n\n// Usage menu\nvoid usage() {\n    fprintf(stderr, "Usage:\\n\\\n    prog [OPTIONS] [ARGS]\\n\\\n\\nARGS:\\n\\\n       0            big-arr\\n\\\n\\n\\\n    OPTIONS:\\n\\\n    -t              (NOT IMPLEMENTED YET) enable time measurement\\n\\n\\\n");\n\n}\n\n\n// -------------------------------

In [29]:
print(env.benchmark.source)

// ========================================================================= //

// includes
#include "stdio.h"
#include "stdlib.h"
#include "time.h"
#include "string.h"
#include "limits.h"
#include "float.h"



#define JOTAI_NUM_RANDS_ 25

const unsigned rand_primes[JOTAI_NUM_RANDS_] = {179, 103, 479, 647, 229, 37, 271, 557, 263, 607, 18743, 50359, 21929, 48757, 98179, 12907, 52937, 64579, 49957, 52567, 507163, 149939, 412157, 680861, 757751};

int next_i() {
  static counter = 0;
  return (-2 * (counter % 2) + 1) * rand_primes[(++counter)%JOTAI_NUM_RANDS_];
}

float next_f() {
  static counter = 0;
  return rand_primes[(++counter)%JOTAI_NUM_RANDS_] / 757751.0F;
} 


// Usage menu
void usage() {
    fprintf(stderr, "Usage:\n\
    prog [OPTIONS] [ARGS]\n\
\nARGS:\n\
       0            big-arr\n\
\n\
    OPTIONS:\n\
    -t              (NOT IMPLEMENTED YET) enable time measurement\n\n\
");

}


// ------------------------------------------------------------------------- //

#define NUL

In [20]:
print(env.observation["Ir"])

; ModuleID = '-'
source_filename = "-"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

%struct._IO_FILE = type { i32, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, %struct._IO_marker*, %struct._IO_FILE*, i32, i32, i64, i16, i8, [1 x i8], i8*, i64, %struct._IO_codecvt*, %struct._IO_wide_data*, %struct._IO_FILE*, i8*, i64, i32, [20 x i8] }
%struct._IO_marker = type opaque
%struct._IO_codecvt = type opaque
%struct._IO_wide_data = type opaque
%struct.TYPE_3__ = type { i64, i64 }

@rand_primes = dso_local constant [25 x i32] [i32 179, i32 103, i32 479, i32 647, i32 229, i32 37, i32 271, i32 557, i32 263, i32 607, i32 18743, i32 50359, i32 21929, i32 48757, i32 98179, i32 12907, i32 52937, i32 64579, i32 49957, i32 52567, i32 507163, i32 149939, i32 412157, i32 680861, i32 757751], align 16
@next_i.counter = internal global i32 0, align 4
@next_f.counter = internal global i32 0, align 4
@stderr 

In [ ]:
def get_ir(source_path, use_bc=False) -> str:
    command = [CLANG_BIN, "-S", "-emit-llvm", source_path]
    proc = subprocess.run(
        command,
        capture_output=True,
        encoding="utf-8",
    )
    return proc.stdout


def get_assembly(ir):
    command = [LLC_BIN]
    proc = subprocess.run(
        command,
        input=ir,
        capture_output=True,
        encoding="utf-8",
    )
    return proc.stdout
    